In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession\
    .builder\
    .appName('test_task')\
    .getOrCreate()

In [53]:
train = spark.read.option("header","true").csv('./data/train.csv')
test = spark.read.option("header","true").csv('./data/test.csv')

In [54]:
cols_to_agg = [col for col in test.columns if 'feature_type' in col]
new_col_names = ['features_type_' + col[-3] +'_stand_' + col[-1] for col in cols_to_agg]
new_col_names

['features_type_1_stand_0',
 'features_type_1_stand_1',
 'features_type_1_stand_2',
 'features_type_1_stand_3',
 'features_type_1_stand_4',
 'features_type_1_stand_5',
 'features_type_1_stand_6',
 'features_type_1_stand_7',
 'features_type_1_stand_8',
 'features_type_1_stand_9']

In [55]:
for col_to_agg, new_col in zip(cols_to_agg, new_col_names):
    avg = test.select(F.avg(F.col(col_to_agg))).collect()[0][0]
    std = test.select(F.stddev(F.col(col_to_agg))).collect()[0][0]
    test = test.withColumn(new_col, (test[col_to_agg] - avg) / std)

In [56]:
cond = "F.when" + ".when".join(["(F.col('" + col + "') == F.col('max_value'), F.lit('" + col[-1] + "'))" for col in new_col_names])
test = test.withColumn("max_value", F.greatest(*new_col_names))\
           .withColumn("max_feature_type_1_index", eval(cond))

In [57]:
test_transformed = test.select(F.col('id_job'), *new_col_names, 'max_feature_type_1_index')

In [58]:
test_transformed.toPandas().to_csv('test_transformed.csv')